In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [76]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential  # ✅ Corrected
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN  # ✅ Corrected
from tensorflow.keras.layers import Dense, Activation, Dropout  # ✅ Corrected
from tensorflow.keras.layers import Embedding  # ✅ Corrected

from tensorflow.keras.utils import to_categorical  # ✅ Corrected
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer  # ✅ Correct
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

In [3]:
import pandas as pd
dtrain = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv')
dval = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
dtest = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')

In [77]:
dtrain

,id,comment_text,toxic,comment_len,comment_pro
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,42,explanation why the edits made under my userna...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,18,daww he matches this background colour im seem...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,42,hey man im really not trying to edit war its j...
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,112,more i cant make any real suggestions on impr...
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,13,you sir are my hero any chance you remember wh...
...,...,...,...,...,...
223544,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,108,jerome i see you never got around to this im n...
223545,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,15,lucky bastard httpwikimediafoundationorgwiki...
223546,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,15,shame on you all you want to speak about ga...
223547,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,27,mel gibson is a nazi bitch who makes shitty mo...


In [5]:
dtest

,id,content,lang
0,0,Doctor Who adlı viki başlığına 12. doctor olar...,tr
1,1,"Вполне возможно, но я пока не вижу необходимо...",ru
2,2,"Quindi tu sei uno di quelli conservativi , ...",it
3,3,Malesef gerçekleştirilmedi ancak şöyle bir şey...,tr
4,4,:Resim:Seldabagcan.jpg resminde kaynak sorunu ...,tr
...,...,...,...
63807,63807,"No, non risponderò, come preannunciato. Prefer...",it
63808,63808,"Ciao, I tecnici della Wikimedia Foundation sta...",it
63809,63809,innnazitutto ti ringrazio per i ringraziamenti...,it
63810,63810,Kaç olumlu oy gerekiyor? Şu an 7 oldu. Hayır...,tr


# Data Preprocessing: 

**Consider this example as binary classification with only toxic**

In [6]:
dtrain = dtrain[['id','comment_text','toxic']]
dtrain.head(5)

,id,comment_text,toxic
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0


In [78]:
each_len = []

for k in dtrain['comment_text']:
    each_len.append(len(k.split(' ')))
    
dtrain['comment_len'] = each_len

In [79]:
dtrain_sorted = dtrain.groupby('comment_len')[['comment_text']].max().sort_values(by='comment_len', ascending=False).reset_index()

dtrain_sorted

,comment_len,comment_text
0,2834,""" \n = = R e g a r d i n g K a r e l = = ..."
1,2681,""" \n\n ==Hey sexy== \n Quit sucking so much ha..."
2,2321,C O M M I E - F U C K E R C O M M I E - F U C ...
3,2273,"""\n\nAs near as I can tell, wikipedia has no a..."
4,2071,== me editing bad. == \n\n well' i want to be ...
...,...,...
1020,5,일이삼사오육칠팔구하고십이요 에헤헤 으헤 으헤 으허허
1021,4,괴로운 변명을 하는 사람이다-_-;;
1022,3,→ English Wikipedia←
1023,2,ขอบคุณครับคำแนะนำครับ ใส่แหล่งอ้างอิงเรียบร้อย...


In [9]:
print(dtrain_sorted.iloc[0]['comment_text'])
print(dtrain_sorted.iloc[0]['comment_len'])

"   
 = = R e g a r d i n g   K a r e l = =  
 
  J o y ,  
 
  I ' m   a   b i t   c o n c e r n e d   o v e r   y o u r   e d i t s   t o   [ [ C h a r l e s   K a r e l   B o u l e y ] ]   o v e r   t h e   p a s t   f e w   m o n t h s ,   a s   y o u r   e d i t s   c o m e   a c r o s s   a s   f a n   e d i t s .     A   n u m b e r   o f   m o r e   c o n t r o v e r s i a l   c o m m e n t s   h a v e   b e e n   r e m o v e d   f r o m   t h e   a r t i c l e   ( g e t t i n g   f i r e d   f r o m   K F I ,   g e t t i n g   i n   t r o u b l e   f o r   h i s   R e a g a n   c o m m e n t s ,   e t c . ) ,   l e a v i n g   t h e   a r t i c l e   c o m e   a c r o s s   w i t h   a   c o m p l e t e l y   f a v o r a b l e   v i e w p o i n t .  
 
  Q u e s t i o n      D o   y o u   k n o w   K a r e l   i n   p e r s o n ?     A r e   y o u   a   f r i e n d   o f   h i s ?     A r e   y o u   a   f a n ?     A   f r e q u e n t   u s e r   o f   h i s   c h a t   r o o

In [80]:
from sklearn.metrics import roc_auc_score, roc_curve, auc

def roc_auc_cal(pred,y):
    a=roc_auc_score(y, pred)
    return a

In [11]:
#EXAMPLE
'''
# Simulated dataset
y_true = [0, 0, 1, 1]
y_scores = [0.1, 0.4, 0.35, 0.8]

print(roc_auc_cal(y_scores,y_true))  ---- 0.75
'''

'\n# Simulated dataset\ny_true = [0, 0, 1, 1]\ny_scores = [0.1, 0.4, 0.35, 0.8]\n\nprint(roc_auc_cal(y_scores,y_true))  ---- 0.75\n'

In [86]:
dtrain

,id,comment_text,toxic,comment_len,comment_pro
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,42,explanation why the edits made under my userna...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,18,daww he matches this background colour im seem...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,42,hey man im really not trying to edit war its j...
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,112,more i cant make any real suggestions on impr...
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,13,you sir are my hero any chance you remember wh...
...,...,...,...,...,...
223544,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,108,jerome i see you never got around to this im n...
223545,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,15,lucky bastard httpwikimediafoundationorgwiki...
223546,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,15,shame on you all you want to speak about ga...
223547,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,27,mel gibson is a nazi bitch who makes shitty mo...


In [84]:
#Converting text to lowercase:
import re
def convt_lower(data):
    tmp = []
    for e in data:
        tmp.append(e.lower())
    return tmp

def remove_specialchar(data):
    tmp = []
    for e in data:
        e = e.replace('\n',' ')
        tmp.append(re.sub(r'[^a-zA-Z0-9\s]', '', e))
    return tmp


In [85]:
comment_lc = convt_lower(dtrain['comment_text'])
comment_punc = remove_specialchar(comment_lc)
dtrain['comment_pro']=comment_punc

**Takeaways here:**

> I need to tokenizer them then i am going to list it into sequence. For doing that, I splitted the data into train and test. But only train data goes to Tokenizer.fit_on_text. If I put train and test data together, tokenization process would be done by considering test data which means the model would see the test data. but it should not see. test data should be unseen.

In [117]:
#first split the data:
y = dtrain['toxic']
x = dtrain['comment_pro']
x_train,x_test,y_train,y_test = train_test_split(x.values,y.values,random_state=42,test_size=0.25, shuffle=True)


#Initialize the tokenizer
token = Tokenizer(num_words=2000, oov_token="<OOV>") 
token.fit_on_texts(x_train)

#Apply the tokenizer here:
x_train_seq = token.texts_to_sequences(x_train) 
x_test_seq = token.texts_to_sequences(x_test)   #Truncation just so that max len of data is 2000 words

#Truncation did not take place before. I will to it now!
for e in x_train_seq:
    if len(e)>2000:
        print(len(e))
        print('blinked!')
        break

x_train_pad = pad_sequences(x_train_seq, maxlen=2000, padding='post', truncating='post') #post or pre
x_test_pad = pad_sequences(x_test_seq, maxlen=2000, padding='post', truncating='post') #post or pre

#now check whether the truncating is done succesfully:
checker = True
for e in x_train_pad:
    if len(e) > 2000:
        print('blinked!')
        checker == False
if checker == True:
    print('truncating is applied sucessfully')
else:
    print('there is a problem in truncating')
    

2142
blinked!
truncating is applied sucessfully
